# Analyzing SOFC data (part 6)

##5) Adding the temperature dimension
D\* and k\* are both thermally activated coefficients, meaning they are always much higher at higher temperatures. Diffusion at low temperatures is much slower because atoms vibrate with lower amplitude and this makes it tougher for ions to flow through the crystals. However, low temperatures are much more practical from an engineering standpoint because the stress on the systems is much lower and mechanical failure takes much longer to arise.

In this notebook I will extend my model, currently only analyzed at 1000K, to lower temperatures and see how well it holds up.